In this notebook, the calculation of risk associated with the variable under examination will be shown. Pandas dataframes will be presented which, in the example, show nine airports - the operator will need to modify the list of airports by adding/modifying names.
The same concept applies to exposure and vulnerability indicators - indicators will be presented that can be expanded or modified by the user.

In [82]:
import os
import pandas as pd
import numpy as np

## **Extreme Temperature:**
## **Exposure and Vulnerability**

In [83]:
# Define the airports
airports = ['Milano Malpensa', 'Milano Linate', 'Bergamo Orio al Serio', 'Roma Fiumicino', 'Roma Ciampino', 
            'Napoli Capodichino', 'Palermo Punta Raisi', 'Catania Fontanarossa', 'Cagliari Elmas']

## **Folder to save the csv file where average exposure and vulnerability are going to be stored**

In [84]:
csv_folder_path = '/work/cmcc/dg07124/climax/indicators/cordex2/vulnerability_exposure'

if not os.path.exists(csv_folder_path):
    os.makedirs(csv_folder_path)

### **Air Land -Side Components**

The various airports components, divided into landside and airside areas, were considered as exposed samples in accordance with De Vivo et al. (2022). More specifically, the air side components include the structures used for the movement of aircraft, such as runways, taxiways, tower, and aprons. On the other hand, the land side components refer to the public access areas such as offices, terminals, airports access systems and parking areas.


| Air side  |Unit of measurement|
|-----------|------------------ |
| Runaway   | Area [m^2]        | 
| Taxiways  | Area [m]          | 
| Tower     | Height [m]        | 
| Apron     | Area [m^2]        | 


--------------------------------------------------------


| Land side                    |Unit of measurement|
|------------------------------|------------------ |
| Terminal                     | Area [m^2]        | 
| Offices and other buildings  | Area [m^2]        | 
| Airport_accesses_systems     | Area [m^2]        | 
| Carparks                     | Number            | 



In [85]:
# Define the values for each indicator and airport
air_land_side_components = {
    'Runways': [470400, 159742, 145530, 396000, 9900, 118260, 292650, 109575, 126000],
    'Taxiways': [250.5, 156, 183, 300, 301, 99, 368, 159, 604],
    'Tower': [80, 47, 30, 56, 60, 40, 25, 12, 28],
    'Apron': [1319000, 387000, 190000, 797250, None, 200000, 148000, 180000, 156000],
    'Terminals': [315000, 85050, 53025, 354300, 20950, 30700, 35400, 43310, 41290],
    'Offices_and_other_buildings': [26715, 13527, 7075, 29520, 5950, 1915, 4180, 6920, 5030],
    'Airport_accesses_systems': [31660, None, None, None, None, None, None, None, None],
    'Carparks': [15000, 3000, 8000, 20100, 1220, 1500, 1364, 1800, 2133]
}

The aim of normalization processes is to transform the values of these indicators, which are measured at different scales and in different units, into comparable values considered on a common scale (Master Adapt, 2018). We used the Min-Max method to transform all values to scores in a range from 0 to 1, where the value 0 represents the optimal level, while the value 1 reflects the most critical estimates:

In [86]:
# Function to normalize data while handling NaN values and ZeroDivisionError
def normalize_data(indicator_values, airports):
    normalized_data = {}

    for indicator, values in indicator_values.items():
        # Filter out None (NaN) values for normalization
        valid_values = [value for value in values if value is not None]

        # Check if we have more than one unique value for normalization
        if len(valid_values) > 1:
            # Get the min and max values for the valid values
            min_val = min(valid_values)
            max_val = max(valid_values)

            # Check for zero division (when min == max)
            if min_val == max_val:
                normalized_values = [None if value is not None else None for value in values]
            else:
                # Normalize the values
                normalized_values = [
                    (value - min_val) / (max_val - min_val) if value is not None else None
                    for value in values
                ]
        else:
            # If all values are the same or all values are NaN, set all normalized values to NaN
            normalized_values = [None if value is not None else None for value in values]

        normalized_data[indicator] = normalized_values

    return normalized_data

In [87]:
# Dictionary to store the dynamic DataFrames
dynamic_dataframes = {}

# Function to display data in a DataFrame format and store the avg_exposure with a unique name
def display_data(indicator_values, airports, df_name):
    # Create a DataFrame from the original data
    df = pd.DataFrame(indicator_values, index=airports)
    print("Original Data:")
    display(df)

    # Normalize the data
    normalized_data = normalize_data(indicator_values, airports)

    # Convert the normalized data to a DataFrame
    normalized_df = pd.DataFrame(normalized_data, index=airports)

    # Calculate the average normalized value for each airport
    normalized_df['average'] = normalized_df.mean(axis=1)
    print("Normalized Data:")
    display(normalized_df)

    # Separate the "Average" column into another DataFrame
    avg_df = normalized_df[['average']]
    print("Avg components:")
    
    # Store the avg_exposure_df_temp in the dynamic_dataframes dictionary with the provided name
    dynamic_dataframes[df_name] = avg_df
    print(f"Stored DataFrame with name: {df_name}")

In [88]:
# Display the air land side components
display_data(air_land_side_components, airports, 'avg_df_exposure')

Original Data:


,Runways,Taxiways,Tower,Apron,Terminals,Offices_and_other_buildings,Airport_accesses_systems,Carparks
Milano Malpensa,470400,250.5,80,1319000.0,315000,26715,31660.0,15000
Milano Linate,159742,156.0,47,387000.0,85050,13527,NaN,3000
Bergamo Orio al Serio,145530,183.0,30,190000.0,53025,7075,NaN,8000
Roma Fiumicino,396000,300.0,56,797250.0,354300,29520,NaN,20100
Roma Ciampino,9900,301.0,60,NaN,20950,5950,NaN,1220
Napoli Capodichino,118260,99.0,40,200000.0,30700,1915,NaN,1500
Palermo Punta Raisi,292650,368.0,25,148000.0,35400,4180,NaN,1364
Catania Fontanarossa,109575,159.0,12,180000.0,43310,6920,NaN,1800
Cagliari Elmas,126000,604.0,28,156000.0,41290,5030,NaN,2133


Normalized Data:


,Runways,Taxiways,Tower,Apron,Terminals,Offices_and_other_buildings,Airport_accesses_systems,Carparks,average
Milano Malpensa,1.000000,0.300000,1.000000,1.000000,0.882106,0.898388,None,0.729873,0.830052
Milano Linate,0.325390,0.112871,0.514706,0.204099,0.192290,0.420648,None,0.094280,0.266326
Bergamo Orio al Serio,0.294528,0.166337,0.264706,0.035867,0.096220,0.186923,None,0.359110,0.200527
Roma Fiumicino,0.838436,0.398020,0.647059,0.554441,1.000000,1.000000,None,1.000000,0.776851
Roma Ciampino,0.000000,0.400000,0.705882,NaN,0.000000,0.146169,None,0.000000,0.208675
Napoli Capodichino,0.235309,0.000000,0.411765,0.044406,0.029249,0.000000,None,0.014831,0.10508
Palermo Punta Raisi,0.614007,0.532673,0.191176,0.000000,0.043348,0.082050,None,0.007627,0.210126
Catania Fontanarossa,0.216450,0.118812,0.000000,0.027327,0.067077,0.181308,None,0.030720,0.09167
Cagliari Elmas,0.252117,1.000000,0.235294,0.006832,0.061017,0.112842,None,0.048358,0.245209


Avg components:
Stored DataFrame with name: avg_df_exposure


## **Exposure calculation**

In [89]:
# Accessing the avg_exposure_temp DataFrames
avg_df_exposure = dynamic_dataframes['avg_df_exposure']
normalized_avg_path = os.path.join(csv_folder_path, f'avg_df_exposure.csv')
avg_df_exposure.to_csv(normalized_avg_path)
display(avg_df_exposure)

,average
Milano Malpensa,0.830052
Milano Linate,0.266326
Bergamo Orio al Serio,0.200527
Roma Fiumicino,0.776851
Roma Ciampino,0.208675
Napoli Capodichino,0.10508
Palermo Punta Raisi,0.210126
Catania Fontanarossa,0.09167
Cagliari Elmas,0.245209


## Calculation of Vulnerability


Vulnerability reflects “the propensity or predisposition of a system to be adversely affected. It encompasses a variety of concepts and elements including sensitivity or susceptibility to harm and lack of capacity to cope and adapt opportunities, or to respond to consequences” (IPCC, 2022).

Vulnerability is composed by sensitivity and adaptive capacity


### **Sensitivity**

Sensitivity is intended as the degree to which the system is (positively or negatively) affected by climate variability or climate change. 

In [23]:
# Define the sensitivity indicators and their values
sensitivity_indicators = {
    'Soil_sealing': [1235, 300, 300, 1590, 133, 217, 391, 225, 246],
    'Passengers': [27000000, 7000000, 13857257, 43532573, 5879496, 10860068, 7018087, 10223113, 4747806],
    'Buildings_bad_conditions_or_maintenance': [None, None, None, None, None, None, None, None, None],
    'Age_buildings': [73, 82, 83, 63, 105, 111, 61, 97, 84],
    'Air_traffic': [201050, 85730, 95377, 309783, 52253, 82577, 54243, 73494, 39691],
    'Parking_accesses': [None, None, None, None, None, None, None, None, None],
    'Staff_work': [1997.1, 855.9, 1044, 2363.9, 1013.1, None, 283, 183, 146]
}

In [24]:
# Execute the function to display the data for Sensitivity Indicators
display_data(sensitivity_indicators, airports, 'avg_df_sensitivity')

Original Data:


,Soil_sealing,Passengers,Buildings_bad_conditions_or_maintenance,Age_buildings,Air_traffic,Parking_accesses,Staff_work
Milano Malpensa,1235,27000000,None,73,201050,None,1997.1
Milano Linate,300,7000000,None,82,85730,None,855.9
Bergamo Orio al Serio,300,13857257,None,83,95377,None,1044.0
Roma Fiumicino,1590,43532573,None,63,309783,None,2363.9
Roma Ciampino,133,5879496,None,105,52253,None,1013.1
Napoli Capodichino,217,10860068,None,111,82577,None,NaN
Palermo Punta Raisi,391,7018087,None,61,54243,None,283.0
Catania Fontanarossa,225,10223113,None,97,73494,None,183.0
Cagliari Elmas,246,4747806,None,84,39691,None,146.0


Normalized Data:


,Soil_sealing,Passengers,Buildings_bad_conditions_or_maintenance,Age_buildings,Air_traffic,Parking_accesses,Staff_work,average
Milano Malpensa,0.756349,0.573735,None,0.24,0.597422,None,0.834618,0.600425
Milano Linate,0.114619,0.058069,None,0.42,0.170457,None,0.320078,0.216644
Bergamo Orio al Serio,0.114619,0.234872,None,0.44,0.206174,None,0.404888,0.280111
Roma Fiumicino,1.000000,1.000000,None,0.04,1.000000,None,1.000000,0.808
Roma Ciampino,0.000000,0.029179,None,0.88,0.046510,None,0.390955,0.269329
Napoli Capodichino,0.057653,0.157594,None,1.00,0.158783,None,NaN,0.343508
Palermo Punta Raisi,0.177076,0.058535,None,0.00,0.053878,None,0.061770,0.070252
Catania Fontanarossa,0.063143,0.141172,None,0.72,0.125154,None,0.016682,0.21323
Cagliari Elmas,0.077557,0.000000,None,0.46,0.000000,None,0.000000,0.107511


Avg components:
Stored DataFrame with name: avg_df_sensitivity


In [25]:
# Accessing the avg_exposure_temp DataFrames
avg_df_sensitivity = dynamic_dataframes['avg_df_sensitivity']
display(avg_df_sensitivity)

,average
Milano Malpensa,0.600425
Milano Linate,0.216644
Bergamo Orio al Serio,0.280111
Roma Fiumicino,0.808
Roma Ciampino,0.269329
Napoli Capodichino,0.343508
Palermo Punta Raisi,0.070252
Catania Fontanarossa,0.21323
Cagliari Elmas,0.107511


## Adaptative capacity

Adaptive capacity is “the ability of systems, institutions, humans and other organisms to adjust to potential damage, to take advantage of

Following the list of Adaptive capacity indicator considered in this work
This indicatori are defined by the classes reported below and included in De Vivo et al. (2021), if the indicator is presence/absence its value will be 0.9/0.1 respectively 


- **Risk awareness: Initiatives for mitigation to climate change:**
 1. Adherence to "NetZero2050" Programme (neutrality3+)
 2. Certification ISO 50001 and initiatives of "energy saving"
 3. Neutrality 3+ Airport carbon accreditation
 4. Neutrality 4+ Airport carbon accreditation; EP-100 intelligent use of energy from "The Climate Group"- Certification ISO 50001
 5. Level 2 Airport carbon accreditation, environmental improvement program
 6. Commitment to obtaining certfiication

**Initiatives for adaptation to climate change:**
- **Efficient drainage system**
 1. Present

- **Insurance policy for extreme events**
 1. Absent

- **Monitoring and alarm system**
 1. Present
 2. Present (but alert wind shear)
 2. Absent
  
- **Bioinfiltration and permeable pavements**
 1. Present
 2. Absent

- **Risk awareness (and initiatives of mitigation)**
 1. Risk management framework 
 2. Development of local (Urban) climate change mitigation and adaptation strategies
 3. Sustainability Report 2019
 4. gesap environmental report
 5. Absent

- **Guidelines for adaptation plan to climate change**
 1. plan (PACC) and regional strategy for  adaptation to climate change
 2. Regional plan under development
 3. Absent
 4. Sustainable Energy and Climate Action Plan (Covenant of Mayors)
 5. Sustainable Energy and Climate Action Plan (Covenant of Mayors) municipality of Catania
 6. Regional climate change adaptation strategy



| Class No. | Description      | Value Rane | Indicator Range (0-1) |
|-----------|------------------|-------------|-----------------------|
| 1         | Optimal          | 0 - 0.2     | 0.1                   |
| 2         | Quite Positive   | 0.2 - 0.4   | 0.3                   |
| 3         | Neutral          | 0.4 - 0.6   | 0.5                   |
| 4         | Quite Negative   | 0.6 - 0.8   | 0.7                   |
| 5         | Critical         | 0.8 - 1     | 0.9                   |


In [26]:
# Define the sensitivity indicators and their values
adaptative_indicators = {
    'Vegetation_zone': [None, None, None, None, None, None, None, None, None],
    'Initiatives_for_optimization_of_energy ': [0.3, 0.3, 0.3, 0.1, 0.1, 0.3, 0.5, 0.7, 0.7],
    'Green_wall': [0.1, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
    'Green_roof': [0.1, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
    'Heat_resistent_coating': [0.9, 0.9, 0.9, 0.1, 0.9, 0.9, 0.9, 0.9, 0.9],
    'Insurance_policy_for_extreme_events': [0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9],
    'Guidelines_for_adaptation_climate_change': [0.3, 0.3, 0.3, 0.5, 0.5, 0.9, 0.3, 0.3, 0.3],
    'Evening departures': [None, None, None, None, None, None, None, None, None]
}

In [27]:
# Create the DataFrame for the adaptive indicators
adaptative_df= pd.DataFrame(adaptative_indicators, index=airports)

# Add a column with the average of all indicators for each airport
adaptative_df['average'] = adaptative_df.mean(axis=1)

display(adaptative_df)

# Extract a column the average column
avg_df_adaptative = adaptative_df[['average']]

display(avg_df_adaptative)

,Vegetation_zone,Initiatives_for_optimization_of_energy,Green_wall,Green_roof,Heat_resistent_coating,Insurance_policy_for_extreme_events,Guidelines_for_adaptation_climate_change,Evening departures,average
Milano Malpensa,None,0.3,0.1,0.1,0.9,0.9,0.3,None,0.433333
Milano Linate,None,0.3,0.9,0.9,0.9,0.9,0.3,None,0.7
Bergamo Orio al Serio,None,0.3,0.9,0.9,0.9,0.9,0.3,None,0.7
Roma Fiumicino,None,0.1,0.9,0.9,0.1,0.9,0.5,None,0.566667
Roma Ciampino,None,0.1,0.9,0.9,0.9,0.9,0.5,None,0.7
Napoli Capodichino,None,0.3,0.9,0.9,0.9,0.9,0.9,None,0.8
Palermo Punta Raisi,None,0.5,0.9,0.9,0.9,0.9,0.3,None,0.733333
Catania Fontanarossa,None,0.7,0.9,0.9,0.9,0.9,0.3,None,0.766667
Cagliari Elmas,None,0.7,0.9,0.9,0.9,0.9,0.3,None,0.766667


,average
Milano Malpensa,0.433333
Milano Linate,0.7
Bergamo Orio al Serio,0.7
Roma Fiumicino,0.566667
Roma Ciampino,0.7
Napoli Capodichino,0.8
Palermo Punta Raisi,0.733333
Catania Fontanarossa,0.766667
Cagliari Elmas,0.766667


## **Vulnerability calculation**
As mentioned before, the Vulnerability is the mean between the sensitivity and adaptive capacity

In [43]:
# Create the vulnerability DataFrame by calculating the average of both 'Average' columns
vulnerability_df = pd.DataFrame()

# Combine the 'Average' columns from both DataFrames and calculate the new average
vulnerability_df['average'] = (avg_df_sensitivity['average'] + avg_df_adaptative['average']) / 2

normalized_avg_path = os.path.join(csv_folder_path, f'avg_df_vulnerability.csv')
vulnerability_df.to_csv(normalized_avg_path)

# Display the resulting vulnerability DataFrame
vulnerability_df

,average
Milano Malpensa,0.516879
Milano Linate,0.458322
Bergamo Orio al Serio,0.490055
Roma Fiumicino,0.687333
Roma Ciampino,0.484664
Napoli Capodichino,0.571754
Palermo Punta Raisi,0.401793
Catania Fontanarossa,0.489948
Cagliari Elmas,0.437089


### Risk Calculation for Extreme Temperature

The risk will be computed by moltiplication of three components: 
R = H * E * V

### UERRA DATASETS:
- **Extreme Temperature**

### **UERRA Dataset**

In the cell below the nearest point to the airport was selected. In the framework the normalization of all the indicator was made, then a mean to obtain a single value

In [48]:
hazard_uerra_temp_path = '/work/cmcc/dg07124/climax/indicators/hazards_csv/hazard_uerra_temp.csv'
hazard_uerra_temp = pd.read_csv(hazard_uerra_temp_path, index_col='Airports')
# Standardize the airport names to Title Case
hazard_uerra_temp.index = hazard_uerra_temp.index.str.title()
hazard_uerra_temp['average'] = pd.to_numeric(hazard_uerra_temp['average'], errors='coerce')
hazard_uerra_temp = hazard_uerra_temp.sort_index()
hazard_uerra_temp

,average
Airports,
Bergamo Orio Al Serio,13.090606
Cagliari Elmas,15.807716
Catania Fontanarossa,17.416528
Milano Linate,13.321168
Milano Malpensa,12.487971
Napoli Capodichino,14.670617
Palermo Punta Raisi,13.622919
Roma Ciampino,14.587898
Roma Fiumicino,12.443477


In [49]:
avg_df_exposure.index = avg_df_exposure.index.str.title()
avg_df_exposure['average'] = pd.to_numeric(avg_df_exposure['average'], errors='coerce')
avg_df_exposure = avg_df_exposure.sort_index()
avg_df_exposure

,average
Bergamo Orio Al Serio,0.200527
Cagliari Elmas,0.245209
Catania Fontanarossa,0.091670
Milano Linate,0.266326
Milano Malpensa,0.830052
Napoli Capodichino,0.105080
Palermo Punta Raisi,0.210126
Roma Ciampino,0.208675
Roma Fiumicino,0.776851


In [50]:
vulnerability_df.index = vulnerability_df.index.str.title()
vulnerability_df['average'] = pd.to_numeric(vulnerability_df['average'], errors='coerce')
vulnerability_df = vulnerability_df.sort_index()
vulnerability_df

,average
Bergamo Orio Al Serio,0.490055
Cagliari Elmas,0.437089
Catania Fontanarossa,0.489948
Milano Linate,0.458322
Milano Malpensa,0.516879
Napoli Capodichino,0.571754
Palermo Punta Raisi,0.401793
Roma Ciampino,0.484664
Roma Fiumicino,0.687333


In [51]:
# Now multiply the 'average' column of each DataFrame
uerra_risk_temp = avg_df_exposure['average'] * hazard_uerra_temp['average'] * vulnerability_df['average']

# Convert the Series to a DataFrame
uerra_risk_temp_df = pd.DataFrame(uerra_risk_temp)

# Rename the column
uerra_risk_temp_df.columns = ['Uerra Risk Temperature']
uerra_risk_temp_df

,Uerra Risk Temperature
Bergamo Orio Al Serio,1.286406
Cagliari Elmas,1.694239
Catania Fontanarossa,0.782242
Milano Linate,1.626026
Milano Malpensa,5.357799
Napoli Capodichino,0.881409
Palermo Punta Raisi,1.150143
Roma Ciampino,1.475383
Roma Fiumicino,6.644263


# **RISK CLASSES**

To evaluate the risk classes for each of the airport the Quantile Method was used. Classifying each airport in terms of quantile with respect to all other airport taken into account

In [52]:
# Function to calculate the quantile of the risk for all 9 airports
def classify_dataframe(df, column_name):
    """
    Classifies a given dataframe column into quantiles and assigns categorical classes.
    
    Parameters:
    df (pd.DataFrame): Input dataframe with numerical values.
    column_name (str): The column name to be classified.
    
    Returns:
    pd.DataFrame: Dataframe with quantile values and class categories.
    """
    df = df.copy()  # Ensure the original dataframe is not modified
    
    # Calculating the quantiles on a scale from 0 to 1
    df['Quantile'] = df[column_name].rank(pct=True)
    
    # Defining classification labels
    labels = ["Very Low", "Low", "Medium", "High", "Very High"]
    
    # Assigning classes based on quantile values
    df['Class'] = pd.qcut(df['Quantile'], q=5, labels=labels)
    return df

In [53]:
# Define a function to color the class column based on the classes
def highlight_class_column(s):
    colors = {
        "Very Low": "background-color: forestgreen",
        "Low": "background-color: limegreen",
        "Medium": "background-color: yellow",
        "High": "background-color: orange",
        "Very High": "background-color: red; color: white"
    }
    return [colors.get(val, "") if s.name == "Class" else "" for val in s]

In [54]:
# Call the function that calculates the quantile
classified_df = classify_dataframe(uerra_risk_temp_df, "Uerra Risk Temperature")

# Apply styling
styled_uerra_risk_temp_df = classified_df.style.apply(highlight_class_column, axis=0)

# Displaying the classified risk
display(styled_uerra_risk_temp_df)

,Uerra Risk Temperature,Quantile,Class
Bergamo Orio Al Serio,1.286406,0.444444,Low
Cagliari Elmas,1.694239,0.777778,High
Catania Fontanarossa,0.782242,0.111111,Very Low
Milano Linate,1.626026,0.666667,High
Milano Malpensa,5.357799,0.888889,Very High
Napoli Capodichino,0.881409,0.222222,Very Low
Palermo Punta Raisi,1.150143,0.333333,Low
Roma Ciampino,1.475383,0.555556,Medium
Roma Fiumicino,6.644263,1.000000,Very High


### **CORDEX Dataset**

The Risk computation for Cordex dataset follows the same steps as UERRA

### RCP26 2021-2050

In [55]:
hazard_cordex_temp_rcp26_20212050_path = '/work/cmcc/dg07124/climax/indicators/cordex2/temp_avg_ensembles/temp_normalized_temp_rcp26_2021-2050_data.csv'
hazard_cordex_temp_rcp26_20212050 = pd.read_csv(hazard_cordex_temp_rcp26_20212050_path, index_col='Airports')
# Standardize the airport names to Title Case
hazard_cordex_temp_rcp26_20212050.index = hazard_cordex_temp_rcp26_20212050.index.str.title()
hazard_cordex_temp_rcp26_20212050['average'] = pd.to_numeric(hazard_cordex_temp_rcp26_20212050['average'], errors='coerce')
hazard_cordex_temp_rcp26_20212050 = hazard_cordex_temp_rcp26_20212050.sort_index()
hazard_cordex_temp_rcp26_20212050 = hazard_cordex_temp_rcp26_20212050[['average']]
hazard_cordex_temp_rcp26_20212050

,average
Airports,
Bergamo Orio Al Serio,0.245054
Cagliari Elmas,0.510344
Catania Fontanarossa,0.792611
Milano Linate,0.369710
Milano Malpensa,0.268374
Napoli Capodichino,0.282604
Palermo Punta Raisi,0.155084
Roma Ciampino,0.388948
Roma Fiumicino,0.320900


In [56]:
# Now multiply the 'average' column of each DataFrame
cordex_risk_temp_rcp26_20212050 = avg_df_exposure['average'] * hazard_cordex_temp_rcp26_20212050['average'] * vulnerability_df['average']
cordex_risk_temp_rcp26_20212050

# Convert the Series to a DataFrame
cordex_risk_temp_rcp26_20212050_df = pd.DataFrame(cordex_risk_temp_rcp26_20212050)

# Rename the column
cordex_risk_temp_rcp26_20212050_df.columns = ['Cordex Risk Temp rcp26 2021-2050']
cordex_risk_temp_rcp26_20212050_df

,Cordex Risk Temp rcp26 2021-2050
Bergamo Orio Al Serio,0.024081
Cagliari Elmas,0.054698
Catania Fontanarossa,0.035599
Milano Linate,0.045128
Milano Malpensa,0.115142
Napoli Capodichino,0.016979
Palermo Punta Raisi,0.013093
Roma Ciampino,0.039337
Roma Fiumicino,0.171347


In [57]:
# Call the function that calculates the quantile
classified_df = classify_dataframe(cordex_risk_temp_rcp26_20212050_df, "Cordex Risk Temp rcp26 2021-2050")

# Apply styling
styled_cordex_risk_temp_df = classified_df.style.apply(highlight_class_column, axis=0)

# Displaying the classified risk
display(styled_cordex_risk_temp_df)

,Cordex Risk Temp rcp26 2021-2050,Quantile,Class
Bergamo Orio Al Serio,0.024081,0.333333,Low
Cagliari Elmas,0.054698,0.777778,High
Catania Fontanarossa,0.035599,0.444444,Low
Milano Linate,0.045128,0.666667,High
Milano Malpensa,0.115142,0.888889,Very High
Napoli Capodichino,0.016979,0.222222,Very Low
Palermo Punta Raisi,0.013093,0.111111,Very Low
Roma Ciampino,0.039337,0.555556,Medium
Roma Fiumicino,0.171347,1.000000,Very High


### RCP26 2041-2070

In [58]:
hazard_cordex_temp_rcp26_20412070_path = '/work/cmcc/dg07124/climax/indicators/cordex2/temp_avg_ensembles/temp_normalized_temp_rcp26_2041-2070_data.csv'
hazard_cordex_temp_rcp26_20412070 = pd.read_csv(hazard_cordex_temp_rcp26_20412070_path, index_col='Airports')
# Standardize the airport names to Title Case
hazard_cordex_temp_rcp26_20412070.index = hazard_cordex_temp_rcp26_20412070.index.str.title()
hazard_cordex_temp_rcp26_20412070['average'] = pd.to_numeric(hazard_cordex_temp_rcp26_20412070['average'], errors='coerce')
hazard_cordex_temp_rcp26_20412070 = hazard_cordex_temp_rcp26_20412070.sort_index()
hazard_cordex_temp_rcp26_20412070 = hazard_cordex_temp_rcp26_20412070[['average']]
hazard_cordex_temp_rcp26_20412070

,average
Airports,
Bergamo Orio Al Serio,0.151568
Cagliari Elmas,0.595888
Catania Fontanarossa,0.824857
Milano Linate,0.214873
Milano Malpensa,0.174961
Napoli Capodichino,0.170859
Palermo Punta Raisi,0.280882
Roma Ciampino,0.366586
Roma Fiumicino,0.234371


In [59]:
# Now multiply the 'average' column of each DataFrame
cordex_risk_temp_rcp26_20412070 = avg_df_exposure['average'] * hazard_cordex_temp_rcp26_20412070['average'] * vulnerability_df['average']
cordex_risk_temp_rcp26_20412070

# Convert the Series to a DataFrame
cordex_risk_temp_rcp26_20412070_df = pd.DataFrame(cordex_risk_temp_rcp26_20412070)

# Rename the column
cordex_risk_temp_rcp26_20412070_df.columns = ['Cordex Risk Temp rcp26 2041-2070']
cordex_risk_temp_rcp26_20412070_df

,Cordex Risk Temp rcp26 2041-2070
Bergamo Orio Al Serio,0.014894
Cagliari Elmas,0.063866
Catania Fontanarossa,0.037047
Milano Linate,0.026228
Milano Malpensa,0.075065
Napoli Capodichino,0.010265
Palermo Punta Raisi,0.023714
Roma Ciampino,0.037076
Roma Fiumicino,0.125144


In [60]:
# Call the function that calculates the quantile
classified_df = classify_dataframe(cordex_risk_temp_rcp26_20412070_df, "Cordex Risk Temp rcp26 2041-2070")

# Apply styling
styled_cordex_risk_temp_df = classified_df.style.apply(highlight_class_column, axis=0)

# Displaying the classified risk
display(styled_cordex_risk_temp_df)

,Cordex Risk Temp rcp26 2041-2070,Quantile,Class
Bergamo Orio Al Serio,0.014894,0.222222,Very Low
Cagliari Elmas,0.063866,0.777778,High
Catania Fontanarossa,0.037047,0.555556,Medium
Milano Linate,0.026228,0.444444,Low
Milano Malpensa,0.075065,0.888889,Very High
Napoli Capodichino,0.010265,0.111111,Very Low
Palermo Punta Raisi,0.023714,0.333333,Low
Roma Ciampino,0.037076,0.666667,High
Roma Fiumicino,0.125144,1.000000,Very High


### RCP26 2071-2100

In [61]:
hazard_cordex_temp_rcp26_20712100_path = '/work/cmcc/dg07124/climax/indicators/cordex2/temp_avg_ensembles/temp_normalized_temp_rcp26_2071-2100_data.csv'
hazard_cordex_temp_rcp26_20712100 = pd.read_csv(hazard_cordex_temp_rcp26_20412070_path, index_col='Airports')
# Standardize the airport names to Title Case
hazard_cordex_temp_rcp26_20712100.index = hazard_cordex_temp_rcp26_20712100.index.str.title()
hazard_cordex_temp_rcp26_20712100['average'] = pd.to_numeric(hazard_cordex_temp_rcp26_20712100['average'], errors='coerce')
hazard_cordex_temp_rcp26_20712100 = hazard_cordex_temp_rcp26_20712100.sort_index()
hazard_cordex_temp_rcp26_20712100 = hazard_cordex_temp_rcp26_20712100[['average']]
hazard_cordex_temp_rcp26_20712100

,average
Airports,
Bergamo Orio Al Serio,0.151568
Cagliari Elmas,0.595888
Catania Fontanarossa,0.824857
Milano Linate,0.214873
Milano Malpensa,0.174961
Napoli Capodichino,0.170859
Palermo Punta Raisi,0.280882
Roma Ciampino,0.366586
Roma Fiumicino,0.234371


In [62]:
# Now multiply the 'average' column of each DataFrame
cordex_risk_temp_rcp26_20712100 = avg_df_exposure['average'] * hazard_cordex_temp_rcp26_20712100['average'] * vulnerability_df['average']
cordex_risk_temp_rcp26_20712100

# Convert the Series to a DataFrame
cordex_risk_temp_rcp26_20712100_df = pd.DataFrame(cordex_risk_temp_rcp26_20712100)

# Rename the column
cordex_risk_temp_rcp26_20712100_df.columns = ['Cordex Risk Temp rcp26 2071-2100']
cordex_risk_temp_rcp26_20712100_df

,Cordex Risk Temp rcp26 2071-2100
Bergamo Orio Al Serio,0.014894
Cagliari Elmas,0.063866
Catania Fontanarossa,0.037047
Milano Linate,0.026228
Milano Malpensa,0.075065
Napoli Capodichino,0.010265
Palermo Punta Raisi,0.023714
Roma Ciampino,0.037076
Roma Fiumicino,0.125144


In [63]:
# Call the function that calculates the quantile
classified_df = classify_dataframe(cordex_risk_temp_rcp26_20712100_df, "Cordex Risk Temp rcp26 2071-2100")

# Apply styling
styled_cordex_risk_temp_df = classified_df.style.apply(highlight_class_column, axis=0)

# Displaying the classified risk
display(styled_cordex_risk_temp_df)

,Cordex Risk Temp rcp26 2071-2100,Quantile,Class
Bergamo Orio Al Serio,0.014894,0.222222,Very Low
Cagliari Elmas,0.063866,0.777778,High
Catania Fontanarossa,0.037047,0.555556,Medium
Milano Linate,0.026228,0.444444,Low
Milano Malpensa,0.075065,0.888889,Very High
Napoli Capodichino,0.010265,0.111111,Very Low
Palermo Punta Raisi,0.023714,0.333333,Low
Roma Ciampino,0.037076,0.666667,High
Roma Fiumicino,0.125144,1.000000,Very High


### RCP45 2021-2050

In [64]:
hazard_cordex_temp_rcp45_20212050_path = '/work/cmcc/dg07124/climax/indicators/cordex2/temp_avg_ensembles/temp_normalized_temp_rcp45_2021-2050_data.csv'
hazard_cordex_temp_rcp45_20212050 = pd.read_csv(hazard_cordex_temp_rcp45_20212050_path, index_col='Airports')
# Standardize the airport names to Title Case
hazard_cordex_temp_rcp45_20212050.index = hazard_cordex_temp_rcp45_20212050.index.str.title()
hazard_cordex_temp_rcp45_20212050['average'] = pd.to_numeric(hazard_cordex_temp_rcp45_20212050['average'], errors='coerce')
hazard_cordex_temp_rcp45_20212050 = hazard_cordex_temp_rcp45_20212050.sort_index()
hazard_cordex_temp_rcp45_20212050 = hazard_cordex_temp_rcp45_20212050[['average']]
hazard_cordex_temp_rcp45_20212050

,average
Airports,
Bergamo Orio Al Serio,0.315651
Cagliari Elmas,0.341100
Catania Fontanarossa,0.889613
Milano Linate,0.441427
Milano Malpensa,0.286262
Napoli Capodichino,0.220931
Palermo Punta Raisi,0.152799
Roma Ciampino,0.438427
Roma Fiumicino,0.187835


In [65]:
# Now multiply the 'average' column of each DataFrame
cordex_risk_temp_rcp45_20212050 = avg_df_exposure['average'] * hazard_cordex_temp_rcp45_20212050['average'] * vulnerability_df['average']
cordex_risk_temp_rcp45_20212050

# Convert the Series to a DataFrame
cordex_risk_temp_rcp45_20212050_df = pd.DataFrame(cordex_risk_temp_rcp45_20212050)

# Rename the column
cordex_risk_temp_rcp45_20212050_df.columns = ['Cordex Risk Temp rcp45 2021-2050']
cordex_risk_temp_rcp45_20212050_df

,Cordex Risk Temp rcp45 2021-2050
Bergamo Orio Al Serio,0.031019
Cagliari Elmas,0.036558
Catania Fontanarossa,0.039956
Milano Linate,0.053882
Milano Malpensa,0.122817
Napoli Capodichino,0.013274
Palermo Punta Raisi,0.012900
Roma Ciampino,0.044341
Roma Fiumicino,0.100295


In [66]:
# Call the function that calculates the quantile
classified_df = classify_dataframe(cordex_risk_temp_rcp45_20212050_df, "Cordex Risk Temp rcp45 2021-2050")

# Apply styling
styled_cordex_risk_temp_df = classified_df.style.apply(highlight_class_column, axis=0)

# Displaying the classified risk
display(styled_cordex_risk_temp_df)

,Cordex Risk Temp rcp45 2021-2050,Quantile,Class
Bergamo Orio Al Serio,0.031019,0.333333,Low
Cagliari Elmas,0.036558,0.444444,Low
Catania Fontanarossa,0.039956,0.555556,Medium
Milano Linate,0.053882,0.777778,High
Milano Malpensa,0.122817,1.000000,Very High
Napoli Capodichino,0.013274,0.222222,Very Low
Palermo Punta Raisi,0.012900,0.111111,Very Low
Roma Ciampino,0.044341,0.666667,High
Roma Fiumicino,0.100295,0.888889,Very High


### RCP45 2041-2070

In [67]:
hazard_cordex_temp_rcp45_20412070_path = '/work/cmcc/dg07124/climax/indicators/cordex2/temp_avg_ensembles/temp_normalized_temp_rcp45_2041-2070_data.csv'
hazard_cordex_temp_rcp45_20412070 = pd.read_csv(hazard_cordex_temp_rcp45_20412070_path, index_col='Airports')
# Standardize the airport names to Title Case
hazard_cordex_temp_rcp45_20412070.index = hazard_cordex_temp_rcp45_20412070.index.str.title()
hazard_cordex_temp_rcp45_20412070['average'] = pd.to_numeric(hazard_cordex_temp_rcp45_20412070['average'], errors='coerce')
hazard_cordex_temp_rcp45_20412070 = hazard_cordex_temp_rcp45_20412070.sort_index()
hazard_cordex_temp_rcp45_20412070 = hazard_cordex_temp_rcp45_20412070[['average']]
hazard_cordex_temp_rcp45_20412070

,average
Airports,
Bergamo Orio Al Serio,0.467816
Cagliari Elmas,0.622728
Catania Fontanarossa,0.727498
Milano Linate,0.548227
Milano Malpensa,0.410582
Napoli Capodichino,0.241711
Palermo Punta Raisi,0.007270
Roma Ciampino,0.428041
Roma Fiumicino,0.244190


In [68]:
# Now multiply the 'average' column of each DataFrame
cordex_risk_temp_rcp45_20412070 = avg_df_exposure['average'] * hazard_cordex_temp_rcp45_20412070['average'] * vulnerability_df['average']
cordex_risk_temp_rcp45_20412070

# Convert the Series to a DataFrame
cordex_risk_temp_rcp45_20412070_df = pd.DataFrame(cordex_risk_temp_rcp45_20412070)

# Rename the column
cordex_risk_temp_rcp45_20412070_df.columns = ['Cordex Risk Temp rcp45 2041-2070']
cordex_risk_temp_rcp45_20412070_df

,Cordex Risk Temp rcp45 2041-2070
Bergamo Orio Al Serio,0.045972
Cagliari Elmas,0.066743
Catania Fontanarossa,0.032675
Milano Linate,0.066918
Milano Malpensa,0.176155
Napoli Capodichino,0.014522
Palermo Punta Raisi,0.000614
Roma Ciampino,0.043291
Roma Fiumicino,0.130387


In [69]:
# Call the function that calculates the quantile
classified_df = classify_dataframe(cordex_risk_temp_rcp45_20412070_df, "Cordex Risk Temp rcp45 2041-2070")

# Apply styling
styled_cordex_risk_temp_df = classified_df.style.apply(highlight_class_column, axis=0)

# Displaying the classified risk
display(styled_cordex_risk_temp_df)

,Cordex Risk Temp rcp45 2041-2070,Quantile,Class
Bergamo Orio Al Serio,0.045972,0.555556,Medium
Cagliari Elmas,0.066743,0.666667,High
Catania Fontanarossa,0.032675,0.333333,Low
Milano Linate,0.066918,0.777778,High
Milano Malpensa,0.176155,1.000000,Very High
Napoli Capodichino,0.014522,0.222222,Very Low
Palermo Punta Raisi,0.000614,0.111111,Very Low
Roma Ciampino,0.043291,0.444444,Low
Roma Fiumicino,0.130387,0.888889,Very High


### RCP45 2071-2100

In [70]:
hazard_cordex_temp_rcp45_20712100_path = '/work/cmcc/dg07124/climax/indicators/cordex2/temp_avg_ensembles/temp_normalized_temp_rcp45_2071-2100_data.csv'
hazard_cordex_temp_rcp45_20712100 = pd.read_csv(hazard_cordex_temp_rcp45_20712100_path, index_col='Airports')
# Standardize the airport names to Title Case
hazard_cordex_temp_rcp45_20712100.index = hazard_cordex_temp_rcp45_20712100.index.str.title()
hazard_cordex_temp_rcp45_20712100['average'] = pd.to_numeric(hazard_cordex_temp_rcp45_20712100['average'], errors='coerce')
hazard_cordex_temp_rcp45_20712100 = hazard_cordex_temp_rcp45_20712100.sort_index()
hazard_cordex_temp_rcp45_20712100 = hazard_cordex_temp_rcp45_20712100[['average']]
hazard_cordex_temp_rcp45_20712100

,average
Airports,
Bergamo Orio Al Serio,0.459278
Cagliari Elmas,0.539877
Catania Fontanarossa,0.838774
Milano Linate,0.574990
Milano Malpensa,0.493102
Napoli Capodichino,0.153590
Palermo Punta Raisi,0.103647
Roma Ciampino,0.228306
Roma Fiumicino,0.081144


In [71]:
# Now multiply the 'average' column of each DataFrame
cordex_risk_temp_rcp45_20712100 = avg_df_exposure['average'] * hazard_cordex_temp_rcp45_20712100['average'] * vulnerability_df['average']
cordex_risk_temp_rcp45_20712100

# Convert the Series to a DataFrame
cordex_risk_temp_rcp45_20712100_df = pd.DataFrame(cordex_risk_temp_rcp45_20712100)

# Rename the column
cordex_risk_temp_rcp45_20712100_df.columns = ['Cordex Risk Temp rcp45 2071-2100']
cordex_risk_temp_rcp45_20712100_df

,Cordex Risk Temp rcp45 2071-2100
Bergamo Orio Al Serio,0.045133
Cagliari Elmas,0.057863
Catania Fontanarossa,0.037673
Milano Linate,0.070185
Milano Malpensa,0.211559
Napoli Capodichino,0.009228
Palermo Punta Raisi,0.008751
Roma Ciampino,0.023090
Roma Fiumicino,0.043327


In [72]:
# Call the function that calculates the quantile
classified_df = classify_dataframe(cordex_risk_temp_rcp45_20712100_df, "Cordex Risk Temp rcp45 2071-2100")

# Apply styling
styled_cordex_risk_temp_df = classified_df.style.apply(highlight_class_column, axis=0)

# Displaying the classified risk
display(styled_cordex_risk_temp_df)

,Cordex Risk Temp rcp45 2071-2100,Quantile,Class
Bergamo Orio Al Serio,0.045133,0.666667,High
Cagliari Elmas,0.057863,0.777778,High
Catania Fontanarossa,0.037673,0.444444,Low
Milano Linate,0.070185,0.888889,Very High
Milano Malpensa,0.211559,1.000000,Very High
Napoli Capodichino,0.009228,0.222222,Very Low
Palermo Punta Raisi,0.008751,0.111111,Very Low
Roma Ciampino,0.023090,0.333333,Low
Roma Fiumicino,0.043327,0.555556,Medium


### RCP85 2021-2050

In [73]:
hazard_cordex_temp_rcp85_20212050_path = '/work/cmcc/dg07124/climax/indicators/cordex2/temp_avg_ensembles/temp_normalized_temp_rcp85_2021-2050_data.csv'
hazard_cordex_temp_rcp85_20212050 = pd.read_csv(hazard_cordex_temp_rcp85_20212050_path, index_col='Airports')
# Standardize the airport names to Title Case
hazard_cordex_temp_rcp85_20212050.index = hazard_cordex_temp_rcp85_20212050.index.str.title()
hazard_cordex_temp_rcp85_20212050['average'] = pd.to_numeric(hazard_cordex_temp_rcp85_20212050['average'], errors='coerce')
hazard_cordex_temp_rcp85_20212050 = hazard_cordex_temp_rcp85_20212050.sort_index()
hazard_cordex_temp_rcp85_20212050 = hazard_cordex_temp_rcp85_20212050[['average']]
hazard_cordex_temp_rcp85_20212050

,average
Airports,
Bergamo Orio Al Serio,0.399005
Cagliari Elmas,0.327102
Catania Fontanarossa,1.000000
Milano Linate,0.576300
Milano Malpensa,0.410556
Napoli Capodichino,0.090410
Palermo Punta Raisi,0.274023
Roma Ciampino,0.264913
Roma Fiumicino,0.108295


In [74]:
# Now multiply the 'average' column of each DataFrame
cordex_risk_temp_rcp85_20212050 = avg_df_exposure['average'] * hazard_cordex_temp_rcp85_20212050['average'] * vulnerability_df['average']
cordex_risk_temp_rcp85_20212050

# Convert the Series to a DataFrame
cordex_risk_temp_rcp85_20212050_df = pd.DataFrame(cordex_risk_temp_rcp85_20212050)

# Rename the column
cordex_risk_temp_rcp85_20212050_df.columns = ['Cordex Risk Temp rcp85 2021-2050']
cordex_risk_temp_rcp85_20212050_df

,Cordex Risk Temp rcp85 2021-2050
Bergamo Orio Al Serio,0.039210
Cagliari Elmas,0.035058
Catania Fontanarossa,0.044914
Milano Linate,0.070345
Milano Malpensa,0.176144
Napoli Capodichino,0.005432
Palermo Punta Raisi,0.023135
Roma Ciampino,0.026793
Roma Fiumicino,0.057825


In [75]:
# Call the function that calculates the quantile
classified_df = classify_dataframe(cordex_risk_temp_rcp85_20212050_df, "Cordex Risk Temp rcp85 2021-2050")

# Apply styling
styled_cordex_risk_temp_df = classified_df.style.apply(highlight_class_column, axis=0)

# Displaying the classified risk
display(styled_cordex_risk_temp_df)

,Cordex Risk Temp rcp85 2021-2050,Quantile,Class
Bergamo Orio Al Serio,0.039210,0.555556,Medium
Cagliari Elmas,0.035058,0.444444,Low
Catania Fontanarossa,0.044914,0.666667,High
Milano Linate,0.070345,0.888889,Very High
Milano Malpensa,0.176144,1.000000,Very High
Napoli Capodichino,0.005432,0.111111,Very Low
Palermo Punta Raisi,0.023135,0.222222,Very Low
Roma Ciampino,0.026793,0.333333,Low
Roma Fiumicino,0.057825,0.777778,High


### RCP85 2041-2070

In [76]:
hazard_cordex_temp_rcp85_20412070_path = '/work/cmcc/dg07124/climax/indicators/cordex2/temp_avg_ensembles/temp_normalized_temp_rcp85_2041-2070_data.csv'
hazard_cordex_temp_rcp85_20412070 = pd.read_csv(hazard_cordex_temp_rcp85_20412070_path, index_col='Airports')
# Standardize the airport names to Title Case
hazard_cordex_temp_rcp85_20412070.index = hazard_cordex_temp_rcp85_20412070.index.str.title()
hazard_cordex_temp_rcp85_20412070['average'] = pd.to_numeric(hazard_cordex_temp_rcp85_20412070['average'], errors='coerce')
hazard_cordex_temp_rcp85_20412070 = hazard_cordex_temp_rcp85_20412070.sort_index()
hazard_cordex_temp_rcp85_20412070 = hazard_cordex_temp_rcp85_20412070[['average']]
hazard_cordex_temp_rcp85_20412070

,average
Airports,
Bergamo Orio Al Serio,0.352040
Cagliari Elmas,0.588853
Catania Fontanarossa,0.865200
Milano Linate,0.488185
Milano Malpensa,0.416816
Napoli Capodichino,0.097882
Palermo Punta Raisi,0.272453
Roma Ciampino,0.288052
Roma Fiumicino,0.086157


In [77]:
# Now multiply the 'average' column of each DataFrame
cordex_risk_temp_rcp85_20412070 = avg_df_exposure['average'] * hazard_cordex_temp_rcp85_20412070['average'] * vulnerability_df['average']
cordex_risk_temp_rcp85_20412070

# Convert the Series to a DataFrame
cordex_risk_temp_rcp85_20412070_df = pd.DataFrame(cordex_risk_temp_rcp85_20412070)

# Rename the column
cordex_risk_temp_rcp85_20412070_df.columns = ['Cordex Risk Temp rcp85 2041-2070']
cordex_risk_temp_rcp85_20412070_df

,Cordex Risk Temp rcp85 2041-2070
Bergamo Orio Al Serio,0.034595
Cagliari Elmas,0.063112
Catania Fontanarossa,0.038859
Milano Linate,0.059589
Milano Malpensa,0.178830
Napoli Capodichino,0.005881
Palermo Punta Raisi,0.023002
Roma Ciampino,0.029133
Roma Fiumicino,0.046004


In [78]:
# Call the function that calculates the quantile
classified_df = classify_dataframe(cordex_risk_temp_rcp85_20412070_df, "Cordex Risk Temp rcp85 2041-2070")

# Apply styling
styled_cordex_risk_temp_df = classified_df.style.apply(highlight_class_column, axis=0)

# Displaying the classified risk
display(styled_cordex_risk_temp_df)

,Cordex Risk Temp rcp85 2041-2070,Quantile,Class
Bergamo Orio Al Serio,0.034595,0.444444,Low
Cagliari Elmas,0.063112,0.888889,Very High
Catania Fontanarossa,0.038859,0.555556,Medium
Milano Linate,0.059589,0.777778,High
Milano Malpensa,0.178830,1.000000,Very High
Napoli Capodichino,0.005881,0.111111,Very Low
Palermo Punta Raisi,0.023002,0.222222,Very Low
Roma Ciampino,0.029133,0.333333,Low
Roma Fiumicino,0.046004,0.666667,High


### RCP85 2071-2100

In [79]:
hazard_cordex_temp_rcp85_20712100_path = '/work/cmcc/dg07124/climax/indicators/cordex2/temp_avg_ensembles/temp_normalized_temp_rcp85_2071-2100_data.csv'
hazard_cordex_temp_rcp85_20712100 = pd.read_csv(hazard_cordex_temp_rcp85_20712100_path, index_col='Airports')
# Standardize the airport names to Title Case
hazard_cordex_temp_rcp85_20712100.index = hazard_cordex_temp_rcp85_20712100.index.str.title()
hazard_cordex_temp_rcp85_20712100['average'] = pd.to_numeric(hazard_cordex_temp_rcp85_20712100['average'], errors='coerce')
hazard_cordex_temp_rcp85_20712100 = hazard_cordex_temp_rcp85_20712100.sort_index()
hazard_cordex_temp_rcp85_20712100 = hazard_cordex_temp_rcp85_20712100[['average']]
hazard_cordex_temp_rcp85_20712100

,average
Airports,
Bergamo Orio Al Serio,0.538966
Cagliari Elmas,0.497329
Catania Fontanarossa,0.799593
Milano Linate,0.684917
Milano Malpensa,0.551032
Napoli Capodichino,0.112875
Palermo Punta Raisi,0.097831
Roma Ciampino,0.330814
Roma Fiumicino,0.161266


In [80]:
# Now multiply the 'average' column of each DataFrame
cordex_risk_temp_rcp85_20712100 = avg_df_exposure['average'] * hazard_cordex_temp_rcp85_20712100['average'] * vulnerability_df['average']
cordex_risk_temp_rcp85_20712100

# Convert the Series to a DataFrame
cordex_risk_temp_rcp85_20712100_df = pd.DataFrame(cordex_risk_temp_rcp85_20712100)

# Rename the column
cordex_risk_temp_rcp85_20712100_df.columns = ['Cordex Risk Temp rcp85 2071-2100']
cordex_risk_temp_rcp85_20712100_df

,Cordex Risk Temp rcp85 2071-2100
Bergamo Orio Al Serio,0.052964
Cagliari Elmas,0.053303
Catania Fontanarossa,0.035913
Milano Linate,0.083603
Milano Malpensa,0.236413
Napoli Capodichino,0.006781
Palermo Punta Raisi,0.008260
Roma Ciampino,0.033458
Roma Fiumicino,0.086109


In [81]:
# Call the function that calculates the quantile
classified_df = classify_dataframe(cordex_risk_temp_rcp85_20712100_df, "Cordex Risk Temp rcp85 2071-2100")

# Apply styling
styled_cordex_risk_temp_df = classified_df.style.apply(highlight_class_column, axis=0)

# Displaying the classified risk
display(styled_cordex_risk_temp_df)

,Cordex Risk Temp rcp85 2071-2100,Quantile,Class
Bergamo Orio Al Serio,0.052964,0.555556,Medium
Cagliari Elmas,0.053303,0.666667,High
Catania Fontanarossa,0.035913,0.444444,Low
Milano Linate,0.083603,0.777778,High
Milano Malpensa,0.236413,1.000000,Very High
Napoli Capodichino,0.006781,0.111111,Very Low
Palermo Punta Raisi,0.008260,0.222222,Very Low
Roma Ciampino,0.033458,0.333333,Low
Roma Fiumicino,0.086109,0.888889,Very High
